# How to pass tool outputs to chat models

:::info Prerequisites
This guide assumes familiarity with the following concepts:

- [Tools](/docs/concepts/#tools)
- [Function/tool calling](/docs/concepts/#functiontool-calling)
- [Using chat models to call tools](/docs/how_to/tool_calling)
- [Defining custom tools](/docs/how_to/custom_tools/)

:::

Some models are capable of **tool calling** - or generating arguments that conform to a specific schema. This guide will demonstrate how to use those tool cals to actually call a function and properly pass the results back to the model.

![Diagram of a tool calling flow](/img/tool_calling_flow.png)

First, let's define our tools and our model:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
  fireworksParams={`model="accounts/fireworks/models/firefunction-v1", temperature=0`}
/>
```

In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass()

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [2]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

llm_with_tools = llm.bind_tools(tools)

Now, let's get the model to call a tool. We'll add it to a list of messages that we'll treat as conversation history:

In [3]:
from langchain_core.messages import HumanMessage

query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages)

print(ai_msg)

messages.append(ai_msg)

content='' additional_kwargs={'tool_calls': [{'id': 'call_nXPLal4Dv5Oykl3INhsKXx9y', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_Kd2LOwBVCDtcbu6M1415lGL6', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 88, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-a5e575d4-98ee-4b9e-8886-a201230295bf-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_nXPLal4Dv5Oykl3INhsKXx9y', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_Kd2LOwBVCDtcbu6M1415lGL6', 'type': 'tool_call'}] usage_metadata={'input_tokens': 88, 'output_tokens': 49, 'total_tokens': 137}


Next let's invoke the tool functions using the args the model populated!

Conveniently, if we invoke a LangChain `Tool` with a `ToolCall`, we'll automatically get back a `ToolMessage` that can be fed back to the model:

:::info Requires `langchain-core >= 0.2.19`

This functionality was added in `langchain-core == 0.2.19`. Please make sure your package is up to date.

If you are on earlier versions of `langchain-core`, you will need to extract the `args` field from the tool and construct a `ToolMessage` manually.

:::

In [4]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nXPLal4Dv5Oykl3INhsKXx9y', 'function': {'arguments': '{"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_Kd2LOwBVCDtcbu6M1415lGL6', 'function': {'arguments': '{"a": 11, "b": 49}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 88, 'total_tokens': 137}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a5e575d4-98ee-4b9e-8886-a201230295bf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_nXPLal4Dv5Oykl3INhsKXx9y', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'call_Kd2LOwBVCDtcbu6M1415lGL6', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 49, 'total_tokens': 137}),
 ToolMessage(content='36', name='multiply', tool_

And finally, we'll invoke the model with the tool results. The model will use this information to generate a final answer to our original query:

In [5]:
llm_with_tools.invoke(messages)

AIMessage(content='3 * 12 is 36 and 11 + 49 is 60.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 153, 'total_tokens': 171}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a2b04fc3-cb38-4c2a-b67f-e2022db9bc8a-0', usage_metadata={'input_tokens': 153, 'output_tokens': 18, 'total_tokens': 171})

Note that each `ToolMessage` must include a `tool_call_id` that matches an `id` in the original tool calls that the model generates. This helps the model match tool responses with tool calls.

Tool calling agents, like those in [LangGraph](https://langchain-ai.github.io/langgraph/tutorials/introduction/), use this basic flow to answer queries and solve tasks.

## Related

- [LangGraph quickstart](https://langchain-ai.github.io/langgraph/tutorials/introduction/)
- Few shot prompting [with tools](/docs/how_to/tools_few_shot/)
- Stream [tool calls](/docs/how_to/tool_streaming/)
- Pass [runtime values to tools](/docs/how_to/tool_runtime)
- Getting [structured outputs](/docs/how_to/structured_output/) from models